<a href="https://colab.research.google.com/github/nurinsalsabl/TUGAS/blob/main/femaledaily1_kelompok2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
from bs4 import BeautifulSoup
import requests as rs
import pandas as pd
import re
import string

# **start**

In [139]:
link = 'https://reviews.femaledaily.com/products/eyes/mascara/maybelline/volum-express-hyper-curl-54?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=newest&page='
page_response = rs.get(link)
page_response

<Response [200]>

In [140]:
soup = BeautifulSoup(page_response.text, 'html.parser')
rev = soup.find('p', {'class': 'text-content'}).text
print (rev)
rekom = soup.find('p', {'class':"recommend"}).text
print (rekom)
tanggal = soup.find('p', {'class':"review-date"}).text
print (tanggal)
reviewer = soup.find('p', {'class':"profile-username"}).text
print (reviewer)


aku udah lama banget pakai mascara ini, dan emang sebagus ituu tidak mudah luntur walaupun terkena air. bahkan nih yaa, pas dibersihin pakai oil cleansing aja masih ada yg sisaa memang se Waterproof itu produk ini, dan kalau buru buru mau pergi pakai dikit aja bulu mata udah baguss
Erarevina_S recommends this product!
a day ago
Erarevina_S


In [141]:
review = []
tanggal = []
reviewer = []

root = 'https://reviews.femaledaily.com/products/eyes/mascara/maybelline/volum-express-hyper-curl-54?cat=&cat_id=0&age_range=&skin_type=&skin_tone=&skin_undertone=&hair_texture=&hair_type=&order=newest&page='


In [142]:

for page in range(1, 100):
    url = rs.get(root + str(page))
    soup = BeautifulSoup(url.text, 'html.parser')

    def scraping(soup):
        ulasan = soup.findAll('div', {'class': 'review-card'})

        for i in ulasan:
            rev = i.find("p", {"class": "text-content"})
            tgl = i.find("p", {"class": "review-date"})
            nama = i.find("p", {"class": "profile-username"})
            if tgl:
                tgl_text = tgl.text.strip()
                if re.search(r'(Jun|Jul|Aug|Sep)\s+2024', tgl_text):
                  tanggal.append(tgl.text.strip() if tgl else None)
                  review.append(rev.text.strip() if rev else None)
                  reviewer.append(nama.text.strip() if nama else None)

    scraping(soup)

In [143]:
df = pd.DataFrame({
    'Tanggal': tanggal,
    'Reviewer': reviewer,
    'Review': review
})

In [144]:
df = df.reindex(columns=['Tanggal', 'Reviewer', 'Review'])
df.to_csv("review_produk.csv", index=False, encoding='utf-8-sig')
print(f'Data berhasil disimpan ke "review_produk.csv". Total halaman yang di-scrap: {page - 1}')

Data berhasil disimpan ke "review_produk.csv". Total halaman yang di-scrap: 98


In [145]:
def preprocess_lowercase(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].str.lower()  # Mengubah teks di kolom menjadi lowercase
        else:
            print(f"Kolom {col} tidak ditemukan di DataFrame.")
    return df



In [146]:
def remove_emoticons(text):
    emoticon_pattern = r'[:;=][oO\-]?[\)DdpP3/\\|o\[\]{}<>]'
    return re.sub(emoticon_pattern, '', text)

In [147]:
def preprocess_remove_emoticons(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].apply(remove_emoticons)  # Menghapus emotikon di kolom asli
        else:
            print(f"Kolom {col} tidak ditemukan di DataFrame.")
    return df

In [153]:
def remove_punctuation(text):
    if isinstance(text, str):
        return re.sub(r'[^a-z\s]', '', text)  # Menghapus semua karakter kecuali huruf, angka, dan spasi
    return text

In [154]:
def preprocess_remove_punctuation(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].apply(remove_punctuation)  # Menghapus punctuation di kolom asli
        else:
            print(f"Kolom {col} tidak ditemukan di DataFrame.")
    return df


In [155]:
stopwords = ['di', 'ke', 'dari', 'pada', 'untuk', 'dengan', 'adalah', 'yang', 'dan', 'atau',
             'jika', 'itu', 'ini', 'tetapi', 'karena','juga', 'akan', 'telah', 'dalam', 'bnr2',
             'soal', 'semua', 'aja', 'doang', 'sama', 'banget', 'apalagi', 'emang',
             'kalo', 'jadi', 'untuk', 'tuh', 'makanya', 'dulu', 'masih', 'pas','sih', 'gitu',
             'menurut', 'ya', 'bikin', 'buat', 'ada']
def preprocess_remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_words)

df['Review'] = df['Review'].apply(lambda x: remove_stopwords(x))

In [156]:
df = pd.DataFrame({
    'Tanggal': tanggal,
    'Reviewer': reviewer,
    'Review': review
})

In [157]:
df = preprocess_lowercase(df, ['Reviewer', 'Review'])
df = preprocess_remove_emoticons(df, ['Review'])
df = preprocess_remove_punctuation(df, ['Review'])

In [158]:
print(df.head)

<bound method NDFrame.head of          Tanggal       Reviewer  \
0    30 Sep 2024        saroh10   
1    30 Sep 2024    sekarrmln24   
2    30 Sep 2024   niethakopa85   
3    29 Sep 2024          shea6   
4    29 Sep 2024     runilaanwr   
..           ...            ...   
421  04 Jun 2024  izzasalsabila   
422  04 Jun 2024     floryartus   
423  04 Jun 2024  mawarsyafitri   
424  03 Jun 2024    shakiladian   
425  02 Jun 2024    ratrigandis   

                                                Review  
0    ini mascaraaaaaa kecintaaan gue bangett sumpaa...  
1    salah satu mascara fav apalagi buat daily kare...  
2    ini tuh maskara pertama yang aku pakai dari pa...  
3    dulu pakai ini pas masih smabuat anak sma masi...  
4    bulumata ku tipe yang lurus terus agak kaku be...  
..                                                 ...  
421  beuhh baru dipake sejam doang bawah mata udah ...  
422  bagus bgt worth to buy harganya juga murah ga ...  
423  maskara yang menambah volume bu

In [159]:
df.to_csv("review_processed.csv", index=False, encoding='utf-8-sig')

In [160]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [161]:
data = {
    'slang': ['ga', 'gak', 'aku', 'gue', 'tp', 'lo', 'banget', 'krn', 'tks', 'yg'],
    'formal': ['tidak', 'tidak', 'saya', 'saya', 'tapi', 'kamu', 'sangat', 'karena', 'terima kasih', 'yang']
}

In [162]:
df_slang = pd.DataFrame(data)
df_slang.to_csv('colloquial_indonesian_lexicon.csv', index=False, encoding='utf-8-sig')
print("File CSV kamus slang telah dibuat!")

File CSV kamus slang telah dibuat!


In [163]:
df_slang = pd.read_csv('colloquial_indonesian_lexicon.csv')

In [165]:
dict_alay = df_slang.set_index('slang')['formal'].to_dict()

In [164]:
def normalize_text(data):
    try:
        word_tokens = word_tokenize(data)
    except Exception as e:
        print(f"Error in tokenization: {e}")
        return data
    result = [dict_alay.get(wt, wt) for wt in word_tokens]
    return ' '.join(result)


In [166]:
df_reviews = pd.read_csv('review_processed.csv')
df_reviews['Review_Normalized'] = df_reviews['Review'].apply(normalize_text)
df_reviews.to_csv("review_produk_normalized.csv", index=False, encoding='utf-8-sig')
print("Data review berhasil dinormalisasi dan disimpan ke 'review_produk_normalized.csv'.")

Data review berhasil dinormalisasi dan disimpan ke 'review_produk_normalized.csv'.
